<a href="https://colab.research.google.com/github/marcusdunn-phd/VideoPostProcessing_PoseEstimation/blob/main/Example_code_for_video_frame_interpolation_and_pose_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Application of video frame interpolation to markerless, single-camera gait analysis**

Dunn et al. (2022)



---



This [Google Colaboratory](https://colab.research.google.com/) notebook serves as a tool to demonstrate video frame interpolation (Huang et al., 2020) and pose estimation (Cao et al., 2021). The implementation of video frame interpolation and pose estimation follows example implementations by Huang et al. (2020) and Cao et al. (2019), respectively. This notebook combines implementations, and allows analysis of multiple videos (*.mp4) located in a [Google Drive](https://www.google.co.uk/intl/en-GB/drive/) folder. Video frame interpolated videos, alongside pose estimation keypoint data, are exported automatically.



---


#Please refer to source code and papers for and troubleshooting:

---

*Video Frame Interpolation:*

Project page: **[Real-Time Intermediate Flow Estimation for Video Frame Interpolation](https://github.com/megvii-research/ECCV2022-RIFE#real-time-intermediate-flow-estimation-for-video-frame-interpolation)**

Citation: *Huang Z, Zhang T, Heng W, Shi B, Zhou, Shuchang. (2020). RIFE: Real-Time Intermediate Flow Estimation for Video Frame Interpolation. arXiv preprint arXiv:2011.06294. https://doi.org/10.48550/arXiv.2011.06294*

---

*OpenPose pose estimation:*

Project page: **[OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose#readme)**

Citation: *Cao Z, Hidalgo G, Simon T, Wei SE, Sheikh Y (2019) OpenPose: Realtime Multi-Person 2D Pose Estimation Using Part Affinity Fields. IEEE Trans Pattern Anal Mach Intell 43(1):172-186. doi:10.1109/TPAMI.2019.2929257*


Map Google Drive

In [ ]:
from google.colab import files

path = '/content/drive/'
from google.colab import drive
drive.mount(path)

Mounted at /content/drive/


Clone RIFE Video Frame Interpolation repository

In [ ]:
!git clone https://github.com/hzwer/arXiv2020-RIFE
!mkdir /content/arXiv2020-RIFE/train_log
%cd /content/arXiv2020-RIFE/train_log
!gdown --id 1APIzVeI-4ZZCEuIRE1m6WYfSCaOsi_7_
!7z e RIFE_trained_model_v3.6.zip

Install RIFE

In [ ]:
%cd /content/arXiv2020-RIFE/
!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
!pip3 install -r requirements.txt

Important: You will need to restart runtime and re-run the previous cell

---



Create the following path and add .mp4 videos
- *'My Drive/VFI_PE_demo/Input/'*

Run the following cell to perform video frame interpolation for videos in:
- *'My Drive/VFI_PE_demo/Input/'*

In [ ]:
import os
import shutil
!nvidia-smi

# define paths
path = '/content/drive/'
input = os.path.join(path + 'My Drive/VFI_PE_demo/Input/')
output = os.path.join(path + 'My Drive/VFI_PE_demo/Output/')

# create folders if needed
if not os.path.exists(input):
  os.mkdir(input)

if not os.path.exists(output):
  os.mkdir(output)

ext = (".mp4", ".MP4")
filenames = []

for filename in os.listdir(input):
    if any([filename.endswith(tuple(ext))]):

      print('Performing VFI: ' + filename)      

      # copy video from Google Drive to root 
      vid  = input + filename
      dst = 'video.mp4'
      shutil.copyfile(vid, dst)

      # VFI
      !python3 inference_video.py --exp=2 --video='video.mp4' --output='VFI.mp4' --UHD

      # rename output
      os.rename('VFI.mp4', 'VFI_' + filename)
      
      # create list of processed videos
      filenames.append('VFI_' + filename)

# copy videos back to Google Drive
for filename in filenames:
  shutil.copyfile(filename, input + filename)


Clone and install OpenPose (this make take some time)

In [ ]:
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  

Run the following cell to perform pose estimation for videos in:
- *'My Drive/VFI_PE_demo/Input/'*

Exported keypoint data and visualised videos are exported to:
- *'My Drive/VFI_PE_demo/Output/'*

In [ ]:
pn = "./openpose/output/"
outPn = "./keypoints/"

# redefine paths if changed
path = '/content/drive/'
input = os.path.join(path + 'My Drive/VFI_PE_demo/Input/')
output = os.path.join(path + 'My Drive/VFI_PE_demo/Output/')

ext = (".mp4", ".MP4")

#data = []
for filename in os.listdir(input): # change folder to VFIin for original videos
    if any([filename.endswith(tuple(ext))]):

      print('Performing pose estimation: ' + filename)      
      
      # create output folder
      if not os.path.exists(pn):
        !mkdir "./openpose/output/"

      #data.append(filename)
      orig = input + filename

      # copy video to root 
      dst = 'video.mp4'
      shutil.copyfile(orig, dst)

      !cd openpose && ./build/examples/openpose/openpose.bin --video ../video.mp4 --write_json ./output/ --display 0  --write_video ../openpose.avi   #Keypoint and video export

      # create output folder
      if not os.path.exists(outPn):
        !mkdir "./keypoints/"

      # copy output data
      for json in os.listdir(pn):
        shutil.copyfile(pn + json, outPn + json)

      # zip output data
      !zip -r outputData.zip ./keypoints/

      # copy output filenames
      name, ext = filename.split('.')

      # copy ZIP data to Google Drive
      zipFN = output + 'outData_' + name + '.zip'
      shutil.copyfile('./outputData.zip', zipFN)

      # convert the result into MP4 for visualisation
      !ffmpeg -y -loglevel info -i openpose.avi output.mp4
      vidFN = output + 'outVideo_' + name + '.mp4'
      shutil.copyfile('./output.mp4', vidFN)

      !rm openpose.avi
      !rm output.mp4

      # clean up
      !rm outputData.zip
      !rm video.mp4
      !rm -rf ./openpose/output/
      !rm -rf ./keypoints/

Unmount Google Drive

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
print('Google Drive unmounted.')

Google Drive unmounted, batch analysis complete.
